In [58]:
# Global Imports
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf

from keras.models import Sequential
from keras.layers import LSTM, Dense, Embedding, Flatten, Dropout, Input
from keras.utils import to_categorical
from tensorflow.keras.models import Model


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder

from keras.optimizers import Adam


import joblib


In [59]:
# Load csv data
data = pd.read_csv('midi_training.csv')

data.head()

,Onset_Beats,Duration_Beats,Midi_Channel,Midi_Pitch,Velocity,Onset_Sec,Duration_Sec,Composer,Piece
0,3.312500,1.380208,0,51,77,1.156250,0.690104,100775a,concerto_1_e_flat_major_1st_2nd_3rd_mvmt_(nc)s...
1,3.364583,1.328125,0,39,76,1.182292,0.664062,100775a,concerto_1_e_flat_major_1st_2nd_3rd_mvmt_(nc)s...
2,3.364583,1.328125,0,63,76,1.182292,0.664062,100775a,concerto_1_e_flat_major_1st_2nd_3rd_mvmt_(nc)s...
3,5.510417,0.130208,0,50,65,2.255208,0.065104,100775a,concerto_1_e_flat_major_1st_2nd_3rd_mvmt_(nc)s...
4,5.526042,0.114583,0,38,71,2.263021,0.057292,100775a,concerto_1_e_flat_major_1st_2nd_3rd_mvmt_(nc)s...


In [60]:
# Convert pitches to tensor
data['Midi_Pitch'] = tf.constant(data['Midi_Pitch'], dtype=tf.int32)

data.head()

,Onset_Beats,Duration_Beats,Midi_Channel,Midi_Pitch,Velocity,Onset_Sec,Duration_Sec,Composer,Piece
0,3.312500,1.380208,0,51,77,1.156250,0.690104,100775a,concerto_1_e_flat_major_1st_2nd_3rd_mvmt_(nc)s...
1,3.364583,1.328125,0,39,76,1.182292,0.664062,100775a,concerto_1_e_flat_major_1st_2nd_3rd_mvmt_(nc)s...
2,3.364583,1.328125,0,63,76,1.182292,0.664062,100775a,concerto_1_e_flat_major_1st_2nd_3rd_mvmt_(nc)s...
3,5.510417,0.130208,0,50,65,2.255208,0.065104,100775a,concerto_1_e_flat_major_1st_2nd_3rd_mvmt_(nc)s...
4,5.526042,0.114583,0,38,71,2.263021,0.057292,100775a,concerto_1_e_flat_major_1st_2nd_3rd_mvmt_(nc)s...


In [61]:
# Binning Velocity

# Number of bins
num_bins = 16
bin_edges = np.linspace(0, 127, num_bins + 1)

# Digitize velocites (convert to bin index)
velocity_bins = np.digitize(data['Velocity'], bin_edges, right=False) - 1

# Convert to tensor
data['Velocity'] = tf.constant(velocity_bins, dtype=tf.int32)

data.head()

,Onset_Beats,Duration_Beats,Midi_Channel,Midi_Pitch,Velocity,Onset_Sec,Duration_Sec,Composer,Piece
0,3.312500,1.380208,0,51,9,1.156250,0.690104,100775a,concerto_1_e_flat_major_1st_2nd_3rd_mvmt_(nc)s...
1,3.364583,1.328125,0,39,9,1.182292,0.664062,100775a,concerto_1_e_flat_major_1st_2nd_3rd_mvmt_(nc)s...
2,3.364583,1.328125,0,63,9,1.182292,0.664062,100775a,concerto_1_e_flat_major_1st_2nd_3rd_mvmt_(nc)s...
3,5.510417,0.130208,0,50,8,2.255208,0.065104,100775a,concerto_1_e_flat_major_1st_2nd_3rd_mvmt_(nc)s...
4,5.526042,0.114583,0,38,8,2.263021,0.057292,100775a,concerto_1_e_flat_major_1st_2nd_3rd_mvmt_(nc)s...


In [62]:
# Log transform Onset
data['Onset_Sec'] = np.log1p(data['Onset_Sec'])

data.head()

,Onset_Beats,Duration_Beats,Midi_Channel,Midi_Pitch,Velocity,Onset_Sec,Duration_Sec,Composer,Piece
0,3.312500,1.380208,0,51,9,0.768371,0.690104,100775a,concerto_1_e_flat_major_1st_2nd_3rd_mvmt_(nc)s...
1,3.364583,1.328125,0,39,9,0.780376,0.664062,100775a,concerto_1_e_flat_major_1st_2nd_3rd_mvmt_(nc)s...
2,3.364583,1.328125,0,63,9,0.780376,0.664062,100775a,concerto_1_e_flat_major_1st_2nd_3rd_mvmt_(nc)s...
3,5.510417,0.130208,0,50,8,1.180256,0.065104,100775a,concerto_1_e_flat_major_1st_2nd_3rd_mvmt_(nc)s...
4,5.526042,0.114583,0,38,8,1.182653,0.057292,100775a,concerto_1_e_flat_major_1st_2nd_3rd_mvmt_(nc)s...


In [63]:
# Clean up duration
data['Duration_Sec'] = data['Duration_Sec'][data['Duration_Sec'] <= 5]

data.head()

,Onset_Beats,Duration_Beats,Midi_Channel,Midi_Pitch,Velocity,Onset_Sec,Duration_Sec,Composer,Piece
0,3.312500,1.380208,0,51,9,0.768371,0.690104,100775a,concerto_1_e_flat_major_1st_2nd_3rd_mvmt_(nc)s...
1,3.364583,1.328125,0,39,9,0.780376,0.664062,100775a,concerto_1_e_flat_major_1st_2nd_3rd_mvmt_(nc)s...
2,3.364583,1.328125,0,63,9,0.780376,0.664062,100775a,concerto_1_e_flat_major_1st_2nd_3rd_mvmt_(nc)s...
3,5.510417,0.130208,0,50,8,1.180256,0.065104,100775a,concerto_1_e_flat_major_1st_2nd_3rd_mvmt_(nc)s...
4,5.526042,0.114583,0,38,8,1.182653,0.057292,100775a,concerto_1_e_flat_major_1st_2nd_3rd_mvmt_(nc)s...


In [64]:
# Binning Durations

# Define binning edges
bins = np.linspace(0, 1, num=6)

# Digitize durations
data['Duration_Sec_test'] = np.digitize(data['Duration_Sec'], bins)



data.head()

,Onset_Beats,Duration_Beats,Midi_Channel,Midi_Pitch,Velocity,Onset_Sec,Duration_Sec,Composer,Piece,Duration_Sec_test
0,3.312500,1.380208,0,51,9,0.768371,0.690104,100775a,concerto_1_e_flat_major_1st_2nd_3rd_mvmt_(nc)s...,4
1,3.364583,1.328125,0,39,9,0.780376,0.664062,100775a,concerto_1_e_flat_major_1st_2nd_3rd_mvmt_(nc)s...,4
2,3.364583,1.328125,0,63,9,0.780376,0.664062,100775a,concerto_1_e_flat_major_1st_2nd_3rd_mvmt_(nc)s...,4
3,5.510417,0.130208,0,50,8,1.180256,0.065104,100775a,concerto_1_e_flat_major_1st_2nd_3rd_mvmt_(nc)s...,1
4,5.526042,0.114583,0,38,8,1.182653,0.057292,100775a,concerto_1_e_flat_major_1st_2nd_3rd_mvmt_(nc)s...,1


In [65]:
normalized_data = data[['Midi_Pitch', 'Velocity', 'Onset_Sec', 'Duration_Sec_test']]

normalized_data.head()

,Midi_Pitch,Velocity,Onset_Sec,Duration_Sec_test
0,51,9,0.768371,4
1,39,9,0.780376,4
2,63,9,0.780376,4
3,50,8,1.180256,1
4,38,8,1.182653,1


In [66]:
# Min Max Scaling
scaler = MinMaxScaler()

normalized_data.loc[:,'Midi_Pitch'] = scaler.fit_transform(normalized_data['Midi_Pitch'].values.reshape(-1,1))
normalized_data.loc[:,'Velocity'] = scaler.fit_transform(normalized_data['Velocity'].values.reshape(-1,1))
normalized_data.loc[:,'Onset_Sec'] = scaler.fit_transform(normalized_data['Onset_Sec'].values.reshape(-1,1))
normalized_data.loc[:,'Duration_Sec_test'] = scaler.fit_transform(normalized_data['Duration_Sec_test'].values.reshape(-1,1))

normalized_data.head()

C:\Users\patk1\AppData\Local\Temp\ipykernel_8576\1287340417.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.36134454 0.2605042  0.46218487 ... 0.57983193 0.59663866 0.60504202]' has dtype incompatible with int32, please explicitly cast to a compatible dtype first.
  normalized_data.loc[:,'Midi_Pitch'] = scaler.fit_transform(normalized_data['Midi_Pitch'].values.reshape(-1,1))
C:\Users\patk1\AppData\Local\Temp\ipykernel_8576\1287340417.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.5625 0.5625 0.5625 ... 0.5    0.5    0.5   ]' has dtype incompatible with int32, please explicitly cast to a compatible dtype first.
  normalized_data.loc[:,'Velocity'] = scaler.fit_transform(normalized_data['Velocity'].values.reshape(-1,1))
C:\Users\patk1\AppData\Local\Temp\ipykernel_8576\1287340417.py:7: FutureWarning: Setting an item of incompatible d

,Midi_Pitch,Velocity,Onset_Sec,Duration_Sec_test
0,0.361345,0.5625,0.098463,0.6
1,0.260504,0.5625,0.100002,0.6
2,0.462185,0.5625,0.100002,0.6
3,0.352941,0.5000,0.151245,0.0
4,0.252101,0.5000,0.151552,0.0


In [72]:
# Sliding Windows

def prepare_sequences(data, sequence_length):
    x = []
    y = []
    for i in range(len(data) - sequence_length):
        # Get the sequence of previous notes
        x_sequence = data[i:i + sequence_length]
        x.append(x_sequence)
        # The target is the next note after the sequence
        y.append(data[i + sequence_length])  # This is the note you're trying to predict
    
    return np.array(x), np.array(y)

# Prepare sequences for training
#x_train, y_train = prepare_sequences(normalized_data.to_numpy(), sequence_length=10)
x, y = prepare_sequences(normalized_data.to_numpy(), sequence_length=10)

# Split into train and test sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [75]:
# Building Keras Model !!!!
# Initialize the model

sequence_length = 10

model = Sequential()

# Input Layer: Explicitly define the input shape
model.add(Input(shape=(sequence_length, 4)))  # 4 features (Midi_Pitch, Velocity, Onset_Sec, Duration_Sec)

# LSTM Layer 1: First LSTM layer, returns sequences to the next LSTM layer
model.add(LSTM(128, return_sequences=True))  # 128 LSTM units
model.add(Dropout(0.3))  # Dropout layer to prevent overfitting

# LSTM Layer 2: Second LSTM layer
model.add(LSTM(64, return_sequences=False))  # This layer does not return sequences
model.add(Dropout(0.3))

# Dense Layer: Output layer that will predict the next note
model.add(Dense(4, activation='sigmoid'))  # Sigmoid for bounded output (in [0, 1] range)

# Compile the model with an optimizer and loss function
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

# Summary of the model architecture
model.summary()


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_6 (LSTM)                   │ (None, 10, 128)        │        68,096 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 10, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_7 (LSTM)                   │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 4)              │           260 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 117,764 (460.02 KB)

 Trainable params: 117,764 (460.02 KB)

 Non-trainable params: 0 (0.00 B)

In [80]:
# Check tensorflow GPU
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [76]:
# Train the model

#history = model.fit(x_train, y_train, epochs=20, batch_size=64, validation_split=0.2)
history1 = model.fit(x_train, y_train, epochs=20, batch_size=64, validation_split=0.2)


Epoch 1/20
11949/66471 ━━━━━━━━━━━━━━━━━━━━ 7:07 8ms/step - accuracy: 0.8304 - loss: 0.0142

KeyboardInterrupt: 

In [74]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(x_test, y_test)

41544/41544 ━━━━━━━━━━━━━━━━━━━━ 99s 2ms/step - accuracy: 0.2663 - loss: 0.0720
